In [27]:
# import gensim
import collections
import random
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.contrib.tensorboard.plugins import projector

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from sklearn.model_selection import train_test_split
import progressbar
from tqdm import tnrange, tqdm_notebook
import scipy
from scipy.spatial.distance import cosine

import altair as alt


In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [17]:
# !mkdir combined_text_20000

In [4]:
LOG_DIR = "combined_text_20000"
path_for_metadata = os.path.join(LOG_DIR,'metadata.tsv')

In [6]:
DATADIR = os.getenv("DATADIR")

In [8]:
labelled = pd.read_csv(os.path.join(DATADIR, 'labelled.csv.gz'), compression='gzip', low_memory=False)

In [10]:
taxon_id_to_base_path = dict(zip(labelled['taxon_id'], labelled['taxon_base_path']))
type(taxon_id_to_base_path)

dict

In [11]:
labelled.content_id.nunique()

202551

In [12]:
labelled['brexit'] = np.where(labelled['level2taxon']=='Brexit', 1, 0)

In [13]:
labelled.brexit.value_counts()

0    298142
1       918
Name: brexit, dtype: int64

In [14]:
corpus_sample = labelled.sample(n=20000, random_state=1234)
corpus = corpus_sample['combined_text'].tolist()

In [35]:
corpus_sample[corpus_sample['taxon_base_path']=='/transport/rail-interoperability']

,base_path,content_id,description,document_type,first_published_at,locale,primary_publishing_organisation,publishing_app,title,body,combined_text,taxon_id,taxon_base_path,taxon_name,level1taxon,level2taxon,level3taxon,level4taxon,level5taxon,brexit
294213,/government/publications/rail-interoperability...,5c7b1355-7631-11e4-a3cb-005056011aef,explains what rail technical specifications fo...,guidance,2011-07-21T00:00:00.000+00:00,en,Department for Transport,whitehall,rail technical specifications for interoperabi...,rail tsis are common harmonised technical stan...,rail technical specifications for interoperabi...,3c33e0e1-b6cf-4caf-a38f-0eb0ad6cd907,/transport/rail-interoperability,Rail interoperability,Transport,Rail,Rail interoperability,NaN,NaN,0


In [15]:
short_corpus=[]
for text in corpus:
    words = text.split()
    truncated = " ".join(words[0:200])
    short_corpus.append(truncated)

In [18]:
corpus_sample['brexit'] = np.where(corpus_sample['level2taxon']=='Brexit', 1, 0) 

In [19]:
corpus_sample.brexit.value_counts()

0    19933
1       67
Name: brexit, dtype: int64

In [20]:
with open(path_for_metadata,'w') as f:
    f.write("Index\tTitle\tTaxon1\tTaxon2\tbrexit\n")
    for index, row in corpus_sample.iterrows():
        f.write("{}\t{}\t{}\t{}\t{}\n".format(index,row['title'], row['level1taxon'],row['level2taxon'], row['brexit']))

In [22]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Reduce logging output.
# tf.logging.set_verbosity(tf.logging.ERROR)
with tf.Session() as session:
    
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    embedded_sentences = session.run(embed(corpus))
#     session.run(embed(corpus))

with tf.Session() as sess:
    # for tensorboard
    emb = tf.Variable(embedded_sentences, name='embedded_sentences')
    sess.run(emb.initializer)
    config = projector.ProjectorConfig()
    summary_writer = tf.summary.FileWriter(LOG_DIR)
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = emb.name

    # Comment out if you don't have metadata
    embedding.metadata_path = path_for_metadata

    projector.visualize_embeddings(summary_writer, config)
    saver = tf.train.Saver([emb])
    saver.save(sess, os.path.join(LOG_DIR, 'combined_text_sample_20000.ckpt'), 1)
    print("Model saved in path: %s" % os.path.join(LOG_DIR, 'combined_text_sample_20000.ckpt'))

INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 38.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 78.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 118.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 158.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 198.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 238.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 278.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 318.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 358.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-enc

In [23]:
taxons = corpus_sample['taxon_id'].unique()

In [24]:
embedded_sentences

array([[ 0.01133383, -0.05098086, -0.00510166, ..., -0.04104903,
         0.04169934, -0.05192604],
       [-0.05254002, -0.04018333,  0.04251332, ...,  0.01300438,
         0.05140104, -0.05542983],
       [-0.05657301,  0.01150932,  0.04566202, ..., -0.04479241,
         0.00847463, -0.06116154],
       ...,
       [-0.03031492, -0.03062237,  0.02877111, ..., -0.00476275,
         0.01316205, -0.04101428],
       [-0.04887617,  0.04477124,  0.01066234, ...,  0.002596  ,
         0.04827252, -0.0509148 ],
       [-0.04850654,  0.01935021,  0.0398589 , ..., -0.05678055,
         0.04869017, -0.0561689 ]], dtype=float32)

In [32]:
taxon_homogeneity = []
for taxon in tqdm_notebook(taxons):
  taxon_embeddings = embedded_sentences[corpus_sample['taxon_id']==taxon]
  taxon_size = taxon_embeddings.shape[0]
  cosine_results = []
  for i in tqdm_notebook(taxon_embeddings):
    for j in taxon_embeddings:
      cosine_results.append(cosine(i, j))
  mean_cosine_for_taxon = np.mean(np.array(cosine_results))
  

  taxon_homogeneity.append([taxon, taxon_size, mean_cosine_for_taxon])
      
  

In [38]:
taxon_homogeneity_df = pd.DataFrame(taxon_homogeneity, columns = ['taxon_id', 'taxon_size', 'mean_cosine_score']).sort_values('mean_cosine_score', ascending=False)
taxon_homogeneity_df['taxon_base_path'] = taxon_homogeneity_df['taxon_id'].map(taxon_id_to_base_path)
taxon_homogeneity_df.shape

(912, 4)

In [39]:
taxon_homogeneity_df[:50]

,taxon_id,taxon_size,mean_cosine_score,taxon_base_path
30,668cd623-c7a8-4159-9575-90caac36d4b4,374,0.591111,/society-and-culture/community-and-society
228,f9e476ef-654d-41ec-97d9-2b6842d4361d,61,0.570916,/society-and-culture/sports-and-leisure
74,68ad6c84-49fd-4871-95bc-5bd48c0f81e1,128,0.561532,/regional-and-local-government/wales
43,b297e49c-7da4-4bc1-8714-da80fa0758d3,163,0.561093,/society-and-culture/equality-rights-and-citiz...
198,52ff5c99-a17b-42c4-a9d7-2cc92cccca39,173,0.558424,/environment/food-and-farming
98,e491505c-77ae-45b2-84be-8c94b94f6a2b,309,0.558315,/defence-and-armed-forces
20,495afdb6-47be-4df1-8b38-91c8adb1eefc,523,0.556866,/business-and-industry
6,8a98b827-82ad-49b4-819e-82c208c551c4,686,0.556201,/government/national-security
19,b29cf14b-54c6-402c-a9f0-77218602d1af,173,0.551378,/society-and-culture/arts-and-culture
247,e2aac912-93c5-4b19-bd3d-9c12676e985c,97,0.546588,/regional-and-local-government/scotland


In [ ]:
alt.Chart(taxon_homogeneity_df).mark_circle(size=60).encode(
    x='taxon_size',
    y='mean_cosine_score',
    tooltip=['taxon_base_path']
).interactive()